<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/training_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Kernel setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
!pip install wandb
!git clone https://github.com/Krankile/npmf.git

In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##General setup

In [4]:
%%capture
!cd npmf && git pull

import math
import multiprocessing
import os
import pickle
from collections import Counter, defaultdict
from dataclasses import asdict, dataclass
from datetime import datetime, timedelta
from operator import itemgetter
from typing import Callable, List, Tuple

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from npmf.utils.colors import main, main2, main3
from npmf.utils.dataset import TimeDeltaDataset
from npmf.utils.dtypes import fundamental_types
from npmf.utils.eikon import column_mapping
from npmf.utils.tests import pickle_df
from npmf.utils.wandb import get_dataset, put_dataset
from numpy.ma.core import outerproduct
from pandas.tseries.offsets import BDay, Day
from sklearn.preprocessing import MinMaxScaler, minmax_scale
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, trange

import wandb as wb

In [5]:
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[main, main2, main3, "black"])
mpl.rcParams['figure.figsize'] = (6, 4)  # (6, 4) is default and used in the paper

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [7]:
np.random.seed(69)

# Create a Neural network class

# Get some data

In [8]:
%%capture
stock_df = get_dataset("stock-oil-final:latest", project="master-test")
fundamentals_df = get_dataset("fundamentals-oil-final:latest", project="master-test")
meta_df = get_dataset("meta-oil-final:latest", project="master-test")
macro_df = get_dataset("macro-oil-final:latest", project="master-test")

stock_df = stock_df.drop_duplicates(subset=["ticker", "date"])

# Run the loop! (Like Odd-Geir Lademo)

![picture](https://drive.google.com/uc?id=1Y55gFQSi4Baovmi0kUQGhbgGOBTI03E7)


In [111]:
class MultivariateNetwork(nn.Module):
    def __init__(self, lag_len, meta_cont_len, meta_cat_len, macro_len, hidden_dim, out_len, **params):
        super().__init__()

        self.lags = nn.Sequential(
            nn.Linear(lag_len, hidden_dim),
            nn.ReLU(),
        )

        self.meta_cont = nn.Sequential(
            nn.Linear(meta_cont_len, hidden_dim),
            nn.ReLU(),
        )

        self.meta_cat = [nn.Embedding(l, hidden_dim) for l in meta_cat_len]

        self.macro = nn.Sequential(
            nn.Linear(macro_len, hidden_dim),
            nn.ReLU(),
        )

        self.predict = nn.Sequential(
            nn.Linear(3*hidden_dim + 9*hidden_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, out_len),
        )


    def forward(self, lags, meta_cont, meta_cat, macro):

        lags = self.lags(lags)
        meta_cont = self.meta_cont(meta_cont)
        meta_cat = torch.cat([emb(meta_cat[:, i]) for i, emb in enumerate(self.meta_cat)], dim=1)
        macro = self.macro(macro)

        x = torch.cat((lags, meta_cont, meta_cat, macro), dim=1)
        x = self.predict(x)

        return x

In [113]:
def mape_loss(target, y_pred):
    mask = ~target.isnan()
    denom = mask.sum(dim=1)
    target[target != target] = 0
    l = ((((y_pred - target).abs() / (target.abs() + 1e-8) * mask)).sum(dim=1) / denom).mean()
    return l

In [114]:
@dataclass
class RunParams:
    n_reports: int
    training_w: int
    forecast_w: int
    epochs: int
    loss_fn: Callable[[torch.Tensor, torch.Tensor], torch.Tensor]

    lag_len: int
    meta_cont_len: int
    meta_cat_len: List[int]
    macro_len: int
    out_len: int
    hidden_dim: int
    batch_size: int

In [122]:
def train(model, optimizer, data_train, data_val, device, params: RunParams, pbar):
    train_losses = []
    val_losses = []

    postfix = dict()
    for epoch in range(params.epochs):
        postfix = {**postfix, "epoch": epoch}
        pbar.set_postfix(postfix)
        for run_type, dataloader in {"train": data_train, "val": data_val}.items():
            model.train(run_type == "train")
            
            for stocks_and_fundamentals, meta_cont, meta_cat, macro, target in dataloader:
                optimizer.zero_grad()
                stocks_and_fundamentals = stocks_and_fundamentals.to(torch.float).to(device)
                meta_cont = meta_cont.to(torch.float).to(device)
                meta_cat = meta_cat.to(torch.long).to(device)
                macro = macro.to(torch.float).to(device)
                target = target.to(torch.float).to(device)

                y_pred = model(stocks_and_fundamentals, meta_cont, meta_cat, macro)
    
                loss = params.loss_fn(target, y_pred)

                if run_type == "train":
                    train_losses.append(loss.item())
                    loss.backward()

                    optimizer.step()
                else:
                    val_losses.append(loss.item())
        postfix = {**postfix, "train_loss": np.mean(train_losses), "val_loss": np.mean(val_losses)}
        pbar.set_postfix(postfix)

    return train_losses, val_losses

In [124]:
def main():
    params = RunParams(
        n_reports=4,
        training_w=240,
        forecast_w=20,
        epochs=10,
        loss_fn=mape_loss,
        lag_len=302,
        meta_cont_len=1,
        meta_cat_len=np.array([89, 5, 70, 185, 1, 3, 5, 10, 44]) + 1, 
        macro_len=1920,
        out_len=20,
        hidden_dim=32,
        batch_size=64,
    )

    cpus = multiprocessing.cpu_count()
    cpus = 0

    model = MultivariateNetwork(**asdict(params))
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

    date_range = pd.date_range(start="2000-12-31", end="2018-10-31", freq="M")
    n_ranges = len(date_range)
    periods = iter(date_range)
    tra = None
    period = next(periods)

    val = TimeDeltaDataset(period, params.training_w, params.forecast_w, params.n_reports, stock_df, fundamentals_df, meta_df, macro_df)

    outer = tqdm(periods, total=(n_ranges-1), desc=f"Period {period.date()}", leave=True, position=0)

    for period in outer:
        outer.set_description(f"Period {period.date()}")
        tra = val
        # TODO Refactor this class to only require the top-level params once
        val = TimeDeltaDataset(period, params.training_w, params.forecast_w, params.n_reports, stock_df, fundamentals_df, meta_df, macro_df)

        tra_loader = DataLoader(tra, params.batch_size, shuffle=True, drop_last=True, num_workers=cpus)
        val_loader = DataLoader(val, params.batch_size, shuffle=False, num_workers=cpus)

        train(model, optimizer, tra_loader, val_loader, device, params, pbar=outer)

main()

Period 2018-10-31: 100%|██████████| 214/214 [33:16<00:00,  9.33s/it, epoch=9, train_loss=0.103, val_loss=0.113]
